In [1]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer, execute
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from math import pi

In [2]:
def qsvm_circuit_for_six_nine(trains, theta):

    q = QuantumRegister(5, name="x")
    c = ClassicalRegister(2, name="c")
    qc = QuantumCircuit(q, c)

    qc.u(-1.57, 0, 0, q[2])    # qc.u3(-1.57, 0, 0, q[2])

    qc.barrier()

    qc.h(q[0])
    qc.h(q[1])

    qc.barrier()

    qc.p(3.17, q[0])    # qc.u1(3.17, q[0])
    qc.p(1.59, q[1])    # qc.u1(1.59, q[1])
    qc.cu(pi, pi/2, -pi/2, 0, q[0], q[2])    # qc.cu3(pi, pi/2, -pi/2, q[0], q[2])
    qc.cu(pi/2, pi/2, -pi/2, 0, q[1], q[2])    # qc.cu3(pi/2, pi/2, -pi/2, q[1], q[2])

    qc.barrier()

    qc.h(q[1])
    qc.cp(-pi/2, q[0], q[1])    # qc.cu1(-pi/2, q[0], q[1])
    qc.h(q[0])

    qc.barrier()

    qc.cry(pi/4, q[0], q[3])
    qc.cry(pi/8, q[1], q[3])

    qc.barrier()

    qc.h(q[0])
    qc.cp(pi/2, q[0], q[1])    # qc.cu1(pi/2, q[0], q[1])
    qc.h(q[1])

    qc.barrier()

    qc.p(-1.59, q[0])    # qc.u1(-1.59, q[1])
    qc.p(-3.17, q[1])    # qc.u1(-3.17, q[0])
    qc.cu(-pi/2, pi/2, -pi/2, 0, q[0], q[2])    # qc.cu3(-pi/2, pi/2, -pi/2, q[0], q[2])
    qc.cu(-pi, pi/2, -pi/2, 0, q[1], q[2])    # qc.cu3(-pi, pi/2, -pi/2, q[1], q[2])

    qc.barrier()

    qc.h(q[0])
    qc.h(q[1])

    qc.barrier()

    qc.x(q[2])
    qc.ccx(q[2], q[3], q[4])
    qc.x(q[2])
    qc.cry(trains[0], q[4], q[1])
    qc.x(q[2])
    qc.ccx(q[2], q[3], q[4])
    qc.x(q[2])

    qc.barrier()

    qc.ccx(q[2], q[3], q[4])
    qc.cry(trains[1], q[4], q[1])
    qc.ccx(q[2], q[3], q[4])

    qc.barrier()

    qc.cry(-theta, q[3], q[1])
    qc.ch(q[3], q[2])

    qc.barrier()

    qc.h(q[3])
    qc.measure(q[3], c[0])
    qc.h(q[0])
    qc.measure(q[0], c[1])
    
    return qc

In [3]:
trains = [0.3194, 2.4174]
thetas = [-0.14415, 2.4519, 2.8465, 0.0499, -0.05998, 2.2331, -0.32316, 2.7939]
counts = []

for theta in thetas:
    qc = qsvm_circuit_for_six_nine(trains, theta)
    # qc.draw(fold=-1)

    simulator = Aer.get_backend('qasm_simulator')
    job = execute(qc, simulator, shots=8192)
    result = job.result().get_counts()
#     print(result)
    result = {key[1]:val for key,val in result.items() if key[0]=='0'}
#     print(result)
    if result['0'] > result['1']:
        counts.append(0)
    else:
        counts.append(1)
    
print(counts)

[0, 1, 1, 0, 0, 1, 0, 1]
